### 목표: RAG Q&A 동작 구축 → 그 다음에 LangGraph 연결 → 마지막에 Memory/ToolNode 확장

#### 2단계: LangGraph 연결
- 1단계에서 만든 RAG QnA를 Class로 정리
- LangGraph Node로 연결 테스트

---

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage
from langgraph.graph.message import add_messages
from langchain_core.prompts import ChatPromptTemplate
from langgraph.prebuilt import ToolNode, tools_condition
from langchain_pinecone import PineconeVectorStore
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langgraph.graph import StateGraph, START, END
from langchain.tools import tool
from functools import partial
from pinecone import Pinecone, ServerlessSpec
from typing import TypedDict, Annotated, Optional, Literal, List
from dotenv import load_dotenv 
import operator
import os



load_dotenv()
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")

pc = Pinecone(api_key=PINECONE_API_KEY)
index_name = "plant-qna"
index = pc.Index(index_name)

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.3)



class ModelQna:
    def __init__(self, llm, tools):
        self.llm = llm
        self.tools = tools
        self.prompt_template = """
        너는 식물에 대해 차분하게 상담해 주는 전문가이다.
        아래 형식을 반드시 지키되, 실제 상담사가 말하듯 자연스럽고 단정적인 말투로 작성한다.
        RAG 검색 정보를 참고하여 상담을 진행한다.
        
        ### RAG 검색 정보 ###
        {context}

        ### 답변 방식 ###
        - 첫 문장은 사용자의 고민에 대한 핵심 답변을 한 줄로 요약한다. (채팅 응답처럼)
        - 이후 이어지는 RAG 정보는 비슷한 사례의 해결 방향을 '요약 3줄'로 정리한다.
        - 모든 문장은 따뜻하지만 과하지 않게, 실제 상담사가 말하듯 단정적으로 말한다.
        - 마지막 문장은 대화를 이어가기 위해 질문형으로 마무리한다.
        
        ### 출력 형식 ###
        [사용자의 상황을 판단해서 가장 핵심적인 조언을 한 문장으로 제시]
        [현재 상황에 맞는 다음 추가 질문 유도]

        사용자 질문: {question}
        """
        self.prompt = ChatPromptTemplate.from_template(self.prompt_template)


    def extract_question(self, messages):
        human_msgs = [m.content for m in messages if isinstance(m, HumanMessage)]
        return human_msgs[-1] if human_msgs else ""


    def build_chain(self):
        return (
            {
                "context": RunnableLambda(lambda q: next(t.run(q) for t in self.tools if t.name == "tool_rag_qna")),
                "question": RunnablePassthrough()
            }
            | self.prompt
            | self.llm
            | StrOutputParser()
        )

    def get_response(self, messages):
        q = self.extract_question(messages)
        chain = self.build_chain()
        response = chain.invoke(q)
        return response



class GraphState(TypedDict):
    messages: Annotated[list, add_messages]   
    current_stage: Literal["collect", "recommend", "qna", "exit"]  
    collected_data: Optional[dict]                               
    recommend_result: Annotated[Optional[List[str]], operator.add]  
    user_action: Literal["None", "Skip", "Continue", "Retry", "Restart", "QnA",]
    picture_exist: Optional[str]

initial_state = {
    "messages": []
}


@tool
def tool_rag_qna(query: str) -> str:
    """식물 상담 QnA 전용 RAG 도구"""
    
    #----- 전역에서 한번만 바꿔야 할듯한
    vector_store = PineconeVectorStore(index=index, embedding=OpenAIEmbeddings(model="text-embedding-3-small"))
    retriever = vector_store.as_retriever(search_kwargs={"k": 3})
    ##-----

    docs = retriever.invoke(query) # retriever(query)
    return "\n\n".join([d.page_content for d in docs])


tools = [tool_rag_qna]
llm_with_tools = llm.bind_tools(tools)


def node_qna(state: GraphState, chatbot: ModelQna):
    response = chatbot.get_response(state["messages"])
    return {
        "messages": [response], # << 단일턴 | 멀티턴 필요하면 >> state["messages"] + [response]
        "current_stage": "qna",
    }


# 마지막 메시지에 tool_calls가 포함돼 있으면 tool_call 노드로 이동. 없으면 done으로 종료
def is_tool_calls(state: GraphState):
    last_message = state["messages"][-1]

    if last_message.tool_calls:
        return "tool_call"
    else:
        return "done"
    

# 툴이 실행된 후 다시 원래 단계(collect/recommend/qna) 로 돌아가도록
def tool_back_to_caller(state: GraphState) -> str:
    current_state = state.get("current_state")

    if current_state and current_state in ["collect", "recommend", "qna"]:
        return current_state
    
    return "done"







# model_qna = ModelQna(llm_with_tools)
model_qna = ModelQna(llm_with_tools, tools)







workflow = StateGraph(GraphState)

workflow.add_node("qna", partial(node_qna, chatbot=model_qna))
workflow.add_node("tools", ToolNode(tools))
workflow.add_conditional_edges("qna", tools_condition)
workflow.add_edge("tools", "qna")
workflow.add_edge(START, "qna")
workflow.add_edge("qna", END)

app = workflow.compile()

- 테스트

In [47]:
test_state = {
    "messages": [
        HumanMessage(content="흙이 너무 축축한데 물을 줄까요?")
    ],
    "current_stage": "qna",
}

result = app.invoke(test_state)
print(result["messages"][-1].content)

In [48]:
result

{'messages': [HumanMessage(content='흙이 너무 축축한데 물을 줄까요?', additional_kwargs={}, response_metadata={}, id='b531886f-a3ef-4c32-a369-361ddd5ffcc4'),
  HumanMessage(content='', additional_kwargs={}, response_metadata={}, id='142a1880-39f9-4435-8068-fb62b472f09a')],
 'current_stage': 'qna'}

In [40]:
test_state = {
    "messages": [
        HumanMessage(content="산세베리아 잎이 노랗고 말랐어요. 물을 줘도 될까요?")
    ],
    "current_stage": "qna",
    "collected_data": None,
    "recommend_result": [],
    "user_action": "None",
    "picture_exist": "None"
}

result = app.invoke(test_state)
print(result["messages"][-1].content)
